In [6]:
import numpy as np
import nnfs

from nnfs.datasets import spiral_data

nnfs.init()

X = [[1,2,3,2.5],
    [2.0, 5.0, -1.0, 2.0],
    [-1.5, 2.7,3.3,-0.8]]


class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
       self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
       self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

class Activation_ReLU:
    def forward (self, inputs):
        self.output = np.maximum(0, inputs)
        
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities 
        #-np.max(inputs) would be a mistake 
        #it would be subtracting the highest value of outputs as inputs are the outputs right now

#the forward method varies depending on which type of loss function is used 
#categoriacal cross entropy is the loss function used 
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)  
        return data_loss
        
#categoriacal cross entropy is the loss function used 
#the Loss_CategoricalCrossEntropy inherits from the Loss class
    
class Loss_CategoricalCrossEntropy(Loss):
    #y_pred are the values from the neural network
    #y_true are the target training values
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        #stops the values being 0 so there are no infinities 
        #1e-7, 1-1e-7 these values are close to 0 and haveing both as a range stops biases
        #clipped
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
    #we want to be able to handle both scalar or one hot encoded values
    #finds out if the passed values are scalar
        if len(y_true.shape) == 1:
            #referencing all the samples in the batch and we use the values in each sample based on the index of y_true
            correct_confidences = y_pred_clipped[range(samples), y_true]
        #for one hot value  
        #multiplying the index of one vector with the element with the same index from the other vector
        #everyhting gets multiplied by 0 except the target classes confidence because of the one hot encoded vectors 
        #then the resulting vectors values are summed together along the sample or same row  
        elif len(y_true.shape) ==2:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        #the vector of values, that comes to sample losses, then take the mean, then return that data loss, the batch loss
        return negative_log_likelihoods
    
X, y = spiral_data(samples=100, classes=3)

#sample input of 2 values 
dense1 = Layer_Dense(2,3)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(3, 3)
activation2 = Activation_Softmax()

dense1.forward(X)
activation1.forward(dense1.output)

dense2.forward(activation1.output)
activation2.forward(dense2.output)

print(activation2.output[:5])
    
loss_function = Loss_CategoricalCrossEntropy()
    #this gives the loss value
loss = loss_function.calculate(activation2.output, y)
print("Loss: ", loss)

#next would be calculating accuracy
#running the outputs by a numpy.arcmax calculation which will return the index value for whichever value is the greatest
#you can also state which batch you want this calculated on
#the accuracy is going to be the mean of the equivalence check of these predicitons
#the predictions being each argmax output from each sample output and the class targets 
#accuracy is practical and useful however, the loss metric is the most useful
#the loss metric is important because it says how wrong something is 
#the goal is to decrease the loss by adjusting the weights and biases of a neural network
#next is the optimisation of weights and biases 

[[0.33333334 0.33333334 0.33333334]
 [0.33331734 0.3333183  0.33336434]
 [0.3332888  0.33329153 0.33341965]
 [0.33325943 0.33326396 0.33347666]
 [0.33323312 0.33323926 0.33352762]]
Loss:  1.098445
